In [0]:
# Set the seed value for the notebook so the results are reproducible
from numpy.random import seed
seed(1)

In [0]:
# Dependencies
import numpy as np
import pandas as pd
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [246]:
import tensorflow
tensorflow.keras.__version__

'2.2.4-tf'

In [247]:
url = 'https://raw.githubusercontent.com/lucyhohler/Final-Project---Data-Analytics/master/Resources/ufo-sightings.csv'
df = pd.read_csv(url, sep=",", na_filter=False)
df.head()

,datetime,city,state,country,shape,duration (seconds),duration (hours/min),comments,date posted,latitude,longitude,month,season,time of day,part of day
0,1/10/1947 20:00,conroe,tx,us,other,4,3 to4 sec.,The date was early 1947 before Arnold&#39s &qu...,4/16/2005,30.311667,-95.455833,1,Winter,8:00 PM,EVENING
1,1/10/1959 18:30,south san francisco,ca,us,disk,120,2 minutes,Its was approximately 6:30pm&#44 I was sitting...,4/27/2004,37.654722,-122.406667,1,Winter,6:30 PM,EVENING
2,1/10/1964 20:00,el cerrito (over old berkeley dump),ca,us,fireball,120,2 minutes,bal of fire in the sky no reported air crashes...,5/15/2006,37.915833,-122.310556,1,Winter,8:00 PM,EVENING
3,1/10/1980 0:00,exeter,nh,us,disk,7800,10min to 2hrs,saw what appeared as very bright star&#44 some...,2/18/2001,42.981389,-70.948333,1,Winter,12:00 AM,NIGHT
4,1/10/1981 20:30,glendora,ca,us,unknown,60,1 minute,On a very foggy night in 1981 I was illuminate...,2/18/2001,34.136111,-117.864444,1,Winter,8:30 PM,EVENING


In [248]:
data = df.drop(columns=["duration (seconds)", "duration (hours/min)", "comments", "latitude", "longitude ", "time of day"])
df1 = df.drop(columns=["datetime", "city", "country", "duration (seconds)", "duration (hours/min)", "comments", "date posted", "latitude", "longitude ", "month", "time of day"])
data.head()

,datetime,city,state,country,shape,date posted,month,season,part of day
0,1/10/1947 20:00,conroe,tx,us,other,4/16/2005,1,Winter,EVENING
1,1/10/1959 18:30,south san francisco,ca,us,disk,4/27/2004,1,Winter,EVENING
2,1/10/1964 20:00,el cerrito (over old berkeley dump),ca,us,fireball,5/15/2006,1,Winter,EVENING
3,1/10/1980 0:00,exeter,nh,us,disk,2/18/2001,1,Winter,NIGHT
4,1/10/1981 20:30,glendora,ca,us,unknown,2/18/2001,1,Winter,EVENING


In [249]:
df1['outcome'] = '1'

import math

for index, row in df1.iterrows():
    outcome = '1' 
    #print(row['shape'])
    if not row['shape'] or row['shape'] == 'other' or row['shape'] == 'unknown':
        outcome = '0'
    row['outcome'] = outcome

df1.head()

,state,shape,season,part of day,outcome
0,tx,other,Winter,EVENING,0
1,ca,disk,Winter,EVENING,1
2,ca,fireball,Winter,EVENING,1
3,nh,disk,Winter,NIGHT,1
4,ca,unknown,Winter,EVENING,0


In [250]:
from sklearn.preprocessing import LabelEncoder

data = data.apply(LabelEncoder().fit_transform)
data.head()

,datetime,city,state,country,shape,date posted,month,season,part of day
0,371,2139,44,0,20,161,0,3,1
1,372,9524,4,0,12,172,0,3,1
2,373,2941,4,0,14,189,0,3,1
3,374,3192,30,0,12,104,0,3,3
4,375,3800,4,0,28,104,0,3,1


In [251]:
# Assign X (data) and y (target)
X = data
y = df1["outcome"]
print(X.shape, y.shape)

(65032, 9) (65032,)


In [0]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.utils import to_categorical

In [0]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=1, stratify=y)

In [0]:
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [0]:
# Step 1: Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

In [0]:
# Step 2: Convert encoded labels to one-hot-encoding
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

In [0]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [0]:
# Create model and add layers
model = Sequential()
model.add(Dense(units=100, activation='relu', input_dim=9))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=2, activation='softmax'))

In [0]:
# Compile and fit the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [260]:
model.summary()

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_36 (Dense)             (None, 100)               1000      
_________________________________________________________________
dense_37 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_38 (Dense)             (None, 2)                 202       
Total params: 11,302
Trainable params: 11,302
Non-trainable params: 0
_________________________________________________________________


In [261]:
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=60,
    shuffle=True,
    verbose=2
)

Epoch 1/60
48774/48774 - 2s - loss: 0.3791 - acc: 0.8489
Epoch 2/60
48774/48774 - 2s - loss: 0.3192 - acc: 0.8630
Epoch 3/60
48774/48774 - 2s - loss: 0.3024 - acc: 0.8610
Epoch 4/60
48774/48774 - 2s - loss: 0.2619 - acc: 0.8650
Epoch 5/60
48774/48774 - 2s - loss: 0.1859 - acc: 0.9213
Epoch 6/60
48774/48774 - 2s - loss: 0.1209 - acc: 0.9809
Epoch 7/60
48774/48774 - 2s - loss: 0.0727 - acc: 0.9964
Epoch 8/60
48774/48774 - 2s - loss: 0.0354 - acc: 0.9997
Epoch 9/60
48774/48774 - 2s - loss: 0.0155 - acc: 1.0000
Epoch 10/60
48774/48774 - 2s - loss: 0.0065 - acc: 1.0000
Epoch 11/60
48774/48774 - 2s - loss: 0.0026 - acc: 1.0000
Epoch 12/60
48774/48774 - 2s - loss: 0.0011 - acc: 1.0000
Epoch 13/60
48774/48774 - 2s - loss: 0.0107 - acc: 0.9975
Epoch 14/60
48774/48774 - 2s - loss: 4.7203e-04 - acc: 1.0000
Epoch 15/60
48774/48774 - 2s - loss: 3.3788e-04 - acc: 1.0000
Epoch 16/60
48774/48774 - 2s - loss: 2.8241e-04 - acc: 1.0000
Epoch 17/60
48774/48774 - 2s - loss: 2.2239e-04 - acc: 1.0000
Epoch 1

In [262]:
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

16258/16258 - 0s - loss: 6.5009e-08 - acc: 1.0000
Normal Neural Network - Loss: 6.500857118014677e-08, Accuracy: 1.0


In [0]:
encoded_predictions = model.predict_classes(X_test_scaled[:20])
prediction_labels = label_encoder.inverse_transform(encoded_predictions)

In [264]:
print(f"Predicted classes: {prediction_labels}")
print(f"Actual Labels: {list(y_test[:20])}")

Predicted classes: ['0' '1' '0' '1' '1' '0' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1'
 '0' '1']
Actual Labels: ['0', '1', '0', '1', '1', '0', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '0', '1']
